## Data Cleaning and Integration

In this notebook, we take raw Bloomberg OHLCV data for S&P 100 stocks and benchmarks and turn it into a clean, consistent dataset ready for analysis and feature engineering. Steps here include standardizing ticker formats, aligning dates, removing duplicates and obvious errors, filling short missing data stretches, and checking for outliers. The final product is a merged file with all stocks and all relevant fields, with problematic rows handled.


## 1. Load Raw Data and Standardize Columns

We load both the master OHLCV file and SPY reference file, then rename all columns so there’s never confusion later on.


In [19]:
import pandas as pd
import re

# Load master file and SPY file
df = pd.read_csv('ohlcv_master.csv', parse_dates=['date'])
spy = pd.read_csv('spy_ohlcv.csv', parse_dates=['date'])

# Standardize column names for both files right away
df = df.rename(columns={'PX_OPEN':'open', 'PX_HIGH':'high', 'PX_LOW':'low', 'PX_LAST':'close', 'PX_VOLUME':'px_volume'})
spy = spy.rename(columns={'px_open':'spy_open', 'px_high':'spy_high', 'px_low':'spy_low', 'px_last':'close_spy', 'px_volume':'spy_volume'})


## 2. Align to SPY Calendar

We filter the stock data so it only includes dates present in SPY. This ensures every row matches a real US trading day and avoids lookahead problems later.


In [20]:
spy_dates = set(spy['date'])
df = df[df['date'].isin(spy_dates)].copy()
print("Date range after alignment:", df['date'].min(), "to", df['date'].max())


Date range after alignment: 2005-04-15 00:00:00 to 2025-04-15 00:00:00


## 3. Remove CUSIPs and Only Keep Real Tickers

We drop any row where the ticker is actually a CUSIP identifier, not a regular stock ticker. Then we strip the ticker to just the root symbol (first word), so it matches what’s used by yfinance and other APIs.


In [21]:
def is_cusip(symbol):
    s = str(symbol).split()[0]
    return bool(re.fullmatch(r'[A-Z0-9]{7,8}', s)) or bool(re.fullmatch(r'[A-Z0-9]{7,8}[DQ]', s))

df = df[~df['ticker'].apply(is_cusip)].copy()
df['final_ticker'] = df['ticker'].apply(lambda x: x.split()[0])


## 4. Keep Only Final Columns for Analysis

We only keep the columns we’ll actually use downstream: date, final_ticker, open, high, low, close, px_volume. This keeps everything clean and lean.


In [22]:
df = df[['date','final_ticker','open','high','low','close','px_volume']]
print("Columns after filtering:", df.columns.tolist())
print(df.head())


Columns after filtering: ['date', 'final_ticker', 'open', 'high', 'low', 'close', 'px_volume']
            date final_ticker     open     high      low    close  px_volume
13106 2005-04-15           AA  66.1789  67.4331  65.2831  65.6190  3797001.0
13107 2005-04-18           AA  65.6414  67.0747  65.3951  66.2685  2941698.0
13108 2005-04-19           AA  66.1341  67.0747  65.0815  65.8206  3017338.0
13109 2005-04-20           AA  65.6638  65.7758  64.3313  64.7232  2400296.0
13110 2005-04-21           AA  65.1263  66.2909  64.3201  66.1789  2780416.0


## 5. Fill in Any Remaining Missing Values

For each ticker, we fill missing prices or volumes forward, then backward. This fills in small gaps and avoids trouble for rolling calculations later.


In [23]:
df = df.sort_values(['final_ticker', 'date'])
cols_to_fill = ['open','high','low','close','px_volume']
df[cols_to_fill] = df.groupby('final_ticker')[cols_to_fill].transform(lambda x: x.ffill().bfill())
print("Missing values after filling:\n", df.isnull().sum())


Missing values after filling:
 date            0
final_ticker    0
open            0
high            0
low             0
close           0
px_volume       0
dtype: int64


## 6. Clean Up Weird Volume Outliers

If any volume entry for a stock is way outside the normal range (1st or 99th percentile), we set it to missing and fill as above. This protects rolling features from being distorted by mistakes.


In [24]:
q_low = df['px_volume'].quantile(0.01)
q_high = df['px_volume'].quantile(0.99)
df.loc[(df['px_volume'] < q_low) | (df['px_volume'] > q_high), 'px_volume'] = None
df['px_volume'] = df.groupby('final_ticker')['px_volume'].transform(lambda x: x.ffill().bfill())


## 7. Remove Duplicate Rows

As a final check, we remove any duplicate [final_ticker, date] pairs.


In [25]:
before = len(df)
df = df.drop_duplicates(subset=['final_ticker', 'date'])
after = len(df)
print(f"Dropped {before - after} duplicate rows.")


Dropped 27240 duplicate rows.


## 8. Sanity Check: Ticker Coverage

We confirm that our ticker column is clean, unique, and ready for modeling or yfinance pulls.


In [26]:
unique_symbols = df['final_ticker'].unique()
print(f"Number of symbols in dataset: {len(unique_symbols)}")
print(f"Sample of tickers: {unique_symbols[:10]}")
if '' in unique_symbols or None in unique_symbols:
    print("Warning: Empty ticker symbol(s) found!")


Number of symbols in dataset: 163
Sample of tickers: ['AA' 'AAPL' 'ABBV' 'ABT' 'ACN' 'ADBE' 'AEP' 'AES' 'AGN' 'AIG']


## 9. Save the Cleaned Data

We save this as our single source of truth for everything else.


In [27]:
df.to_csv('ohlcv_master_clean.csv', index=False)
